In [ ]:
# Essential imports
import re

from io import StringIO

import pandas as pd
import numpy as np

from Bio import SeqIO
from bioservices import UniProt
from bioservices import Pfam

import tqdm
from pathlib import Path

# We'll need ETE3 to parse and render the tree
from ete3 import Tree, TreeStyle, NodeStyle, TextFace, CircleFace, faces, SeqMotifFace
from tqdm.notebook import tqdm
import pandas as pd
# Connect to UniProt service
u = UniProt(verbose=False)

In [245]:
#!/usr/bin/env python3

import json
import ssl
import time
from urllib import request
from urllib.error import HTTPError

BASE_URL = "https://www.ebi.ac.uk/interpro/api/entry/pfam/protein/UniProt/"

def fetch_interpro_data(uniprotid):
    """Query InterPro API for a single UniProt ID and return results as a list"""

    context = ssl._create_unverified_context()
    next_url = f"{BASE_URL}{uniprotid}"
    last_page = False
    results = []

    attempts = 0
    while next_url:
        try:
            req = request.Request(next_url, headers={"Accept": "application/json"})
            res = request.urlopen(req, context=context)

            if res.status == 408:
                time.sleep(61)
                continue
            elif res.status == 204:
                break

            payload = json.loads(res.read().decode())
            next_url = payload.get("next", None)
            attempts = 0

            if not next_url:
                last_page = True

        except HTTPError as e:
            if e.code == 408:
                time.sleep(61)
                continue
            elif attempts < 3:
                attempts += 1
                time.sleep(61)
                continue
            else:
                print(f"ERROR: {uniprotid} - {e}")
                return []

        # Append results for this UniProt ID
        results.extend(payload["results"])

        if next_url:
            time.sleep(1)  # Prevent API overload

    return results

# List of UniProt IDs to search
# extradomains = ["P69905", "P68871", "Q9Y2Z9", "P05067"]  # Replace with your actual IDs

# Dictionary to store results
interpro_results = {}

# Loop through each UniProt ID and store results
for uniprotid in extradomains:
    interpro_results[uniprotid] = fetch_interpro_data(uniprotid)

# Now interpro_results is a dictionary with UniProt IDs as keys and InterPro data as values

# Print the first few results for testing
for uniprotid, data in interpro_results.items():
    print(f"{uniprotid}: {data[:2]}")  # Show only first 2 entries per ID for readability

# Optionally, save results to a JSON file
with open("interpro_results.json", "w") as f:
    json.dump(interpro_results, f, indent=4)


A0A066ZRQ5: [{'metadata': {'accession': 'PF00909', 'name': 'Ammonium Transporter Family', 'source_database': 'pfam', 'type': 'family', 'integrated': 'IPR024041', 'member_databases': None, 'go_terms': None}, 'proteins': [{'accession': 'a0a066zrq5', 'protein_length': 611, 'source_database': 'unreviewed', 'organism': '28885', 'in_alphafold': True, 'entry_protein_locations': [{'fragments': [{'start': 9, 'end': 385, 'dc-status': 'CONTINUOUS'}], 'representative': False, 'model': 'PF00909', 'score': 4.5e-101}]}]}, {'metadata': {'accession': 'PF00990', 'name': 'Diguanylate cyclase, GGDEF domain', 'source_database': 'pfam', 'type': 'domain', 'integrated': 'IPR000160', 'member_databases': None, 'go_terms': None}, 'proteins': [{'accession': 'a0a066zrq5', 'protein_length': 611, 'source_database': 'unreviewed', 'organism': '28885', 'in_alphafold': True, 'entry_protein_locations': [{'fragments': [{'start': 448, 'end': 606, 'dc-status': 'CONTINUOUS'}], 'representative': False, 'model': 'PF00990', 'sc

In [13]:
interpro_results_extra

{'A0A377PLP5': [{'metadata': {'accession': 'PF00909',
    'name': 'Ammonium Transporter Family',
    'source_database': 'pfam',
    'type': 'family',
    'integrated': 'IPR024041',
    'member_databases': None,
    'go_terms': None},
   'proteins': [{'accession': 'a0a377plp5',
     'protein_length': 430,
     'source_database': 'unreviewed',
     'organism': '569',
     'in_alphafold': True,
     'entry_protein_locations': [{'fragments': [{'start': 35,
         'end': 428,
         'dc-status': 'CONTINUOUS'}],
       'representative': False,
       'model': 'PF00909',
       'score': 2.3e-123}]}]}],
 'A0A097R891': [{'metadata': {'accession': 'PF00909',
    'name': 'Ammonium Transporter Family',
    'source_database': 'pfam',
    'type': 'family',
    'integrated': 'IPR024041',
    'member_databases': None,
    'go_terms': None},
   'proteins': [{'accession': 'a0a097r891',
     'protein_length': 429,
     'source_database': 'unreviewed',
     'organism': '1453496',
     'in_alphafold': 

In [14]:
domain_info = []
name = []
length = []
sted = []
domain_dictionary ={}
for itom in idsfromtree:

    for item in interpro_results_extra[itom]:
        # domain_id = item["metadata"]["proteins"]["accession"]  # Extract domain ID
        domain_name = item["metadata"]["name"]  # Extract domain name
        name.append(domain_name)
        print(domain_name)
        protein_lenth = item["proteins"][0]["protein_length"]
        length.append(protein_lenth)
        print(protein_lenth)
        str_end = item["proteins"][0]['entry_protein_locations'][0]["fragments"]
        sted.append(str_end)
        print(str_end)
        domain_info.append((f"{itom}: {domain_name} {str_end} {protein_lenth}"))
        domain_dictionary[str(itom)] = (domain_name, name, str_end, protein_lenth)


Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
429
[{'start': 34, 'end': 427, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
429
[{'start': 34, 'end': 427, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
429
[{'start': 34, 'end': 427, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
430
[{'start': 35, 'end': 428, 'dc-status': 'CONTINUOUS'}]
Ammonium Transporter Family
427
[{'start': 

In [325]:
domain_dictionary

{'A0A066ZRQ5': ('Diguanylate cyclase, GGDEF domain',
  ['Ammonium Transporter Family',
   'Diguanylate cyclase, GGDEF domain',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'His Kinase A (phospho-acceptor) domain',
   'Ammonium Transporter Family',
   'Histidine kinase-, DNA gyrase B-, and HSP90-like ATPase',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Ammonium Transporter Family',
   'NPL4 family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory protein P-II',
   'Ammonium Transporter Family',
   'Nitrogen regulatory

In [15]:
new_dom_dict_extra = {}

for leaf in tr:
    for t in domain_info:
        if leaf.name.split(' ')[-1] == t.split(': ')[0]:
            s = t.split(': ')[1:]
            new_dom_dict_extra[str(leaf)] = s
            

In [16]:
len(new_dom_dict_extra)

1299

In [1]:
new_dom_dict_extra

NameError: name 'new_dom_dict_extra' is not defined

In [344]:
with open('new_dom_dict_extra.txt', 'w') as fp:
    for ite in new_dom_dict_extra:
        # write each item on a new line
        fp.write("%s\n" % ite)
    print('Done')

Done


In [205]:
u.search('A0A066ZRQ5', frmt = 'tsv', columns = "ft_domain")

'Domain [FT]\nDOMAIN 478..611; /note="GGDEF"; /evidence="ECO:0000259|PROSITE:PS50887"\n'

In [10]:
treepath = Path("EYROOTED.tree")  # transporter family tree, Newick format
annopath = Path("all_annotations.csv")     # annotations for some family members
stem = "st_split"
tr = Tree('EYROOTED.tree')

In [19]:
# Load the tree
# We have these as functions so that we can annotate/render a clean tree each time
def load_tree():
    tree = Tree(str(treepath))
    return tree

# Load the annotation
def load_annotation():
    anno = pd.read_csv(annopath)
    return anno

# The function lets us return the tree and its corresponding annotation,
# where the leaves of this specific tree object are present in the
# annotation dataframe, and so can be manipulated easily.
def annotate_tree():
    tree = load_tree()
    anno = load_annotation()

    leaves = []  # Will hold leaves for the tree where they match the annotation row, or None if there is none
    
    for id in anno["0"]:               # iterate over annotations
        for leaf in tree.iter_leaves():  # iterate over all leaves in the tree
            assigned = False
            if id in str(leaf):
                leaves.append(leaf)
                assigned = True
                break
        if not assigned:
            leaves.append(None)
    
    anno["leaves"] = leaves

    return tree, anno

In [275]:
# # Key by UniProt ID, values are the leaf, ft_domain information, and sequence length as
# # (leaf, ft_info, length)
# domain_dict = {}

# # Iterate over leaves and query UniProt
# # Assume that the last part of the string is the accession ID at UniProt
# # (we also need to strip the final quote)
# for leaf in tqdm(tree.iter_leaves()):
accession = str(leaf).split()[-1][:-1]
#     result = u.search(accession, frmt = 'tsv', columns = "ft_domain")
#     reslen = u.search(accession, frmt = 'tsv', columns = "length")
#     if (reslen is None) or (reslen.strip() == "Length"):
#         length = None
#     else:
#         length = int(reslen.split()[-1].strip())
#     domain_dict[str(leaf)] = (accession, result, length)

In [246]:
u.search('A0A066ZRQ5', frmt = 'tsv', columns = "ft_domain")


"A0A833GZR2: Ammonium Transporter Family [{'start': 9, 'end': 401, 'dc-status': 'CONTINUOUS'}] 681",

'Domain [FT]\nDOMAIN 478..611; /note="GGDEF"; /evidence="ECO:0000259|PROSITE:PS50887"\n'

In [352]:
for doms in domain_info:
    id = doms.split(':')[0]
    reslen = u.search(id, frmt = 'tsv', columns = "length")
    if (reslen is None) or (reslen.strip() == "Length"):
        length = None
    else:
        length = int(reslen.split()[-1].strip())

In [21]:
def result_to_face(data, fontsize=2):
    """Return a SeqMotifFace, based on the passed UniProt result"""
    # domain_name, str_end, protein_lenth = data  # split tuple
    motifs = []
    # data = sted.split("\n")[-1].strip()
    domains = domain_info
    for domain in domains:
        e = domain.split(", 'dc-st")[0]
        # print(e)
        end = int(e.split("d': ")[1])
        st = e.split("rt': ")[1]
        start = int(st.split(", 'e")[0])
        # start, end = tuple([int(_) for _ in se[0].split(", 'end': ")])
        domdata = domain.split(' [')[0]
        name = domdata.split(": ")[1]
        # seq.start, seq.end, shape, width, height, fgcolor, bgcolor
        if name == "Ammonium transporter Family":
            motifs.append([start, end, "[]", None, 10, "black", "blue", f"arial|{fontsize}|white|{name}"])
        else:
            motifs.append([start, end, "[]", None, 10, "black", "red", f"arial|{fontsize}|white|{name}"])
        
        reslen = domain.split(' ')[-1]
        if (reslen is None) or (reslen.strip() == "Length"):
            length = None
    else:
        length = int(reslen.split()[-1].strip())
    if len(motifs):
        if length is None:
            return SeqMotifFace(seq=None, motifs=motifs, seq_format="-")
        else:
            return SeqMotifFace(seq="-" * length, motifs=motifs, seq_format="-")
    return None

In [ ]:
tree, anno = annotate_tree()  # get clean tree

# Declare tree style
everything = TreeStyle()
everything.show_leaf_name = False
everything.mode = "c"

# One text face for each kingdom
face_dict = {"Eukaryota": TextFace("eukaryota"),
             "Bacteria": TextFace("bacteria"),
             "Archaea": TextFace("archaea")}

# Set colours for kingdoms
colour_dict = {"Eukaryota": "#FFFACD", "Bacteria": "#F0F8FF", "Archaea": "#FFE4E1"}

# Set face colours for kingdoms
colour_dict = {"Eukaryota": "#FFFACD", "Bacteria": "#F0F8FF", "Archaea": "#FFE4E1"}

# Set colours for residue types
rescolours = {"Y": "#FFCC66", "E": "#009933", "M": "#9966FF"}

# Iterate over leaves
for idx, row in tqdm(anno.iterrows()):
    # Get leaf information
    leaf = row["leaves"]
    restype = row["first"]
    fourres = row["four"]    
    kingdomname = row["kingdom"].strip()    

    # Style the leaf node
    leaf.img_style["bgcolor"] = colour_dict[kingdomname]    
    leaf.add_face(face_dict[kingdomname], 1, "aligned")

    # Add gateway residue bubble
    if restype in ("Y", "E", "M"):
        face = CircleFace(radius=20, color=rescolours[restype], style="sphere", label=restype)
        face.opacity = 0.3
        leaf.add_face(face, 1, position="float")

    # Add four residues label
    if len(fourres):
        leaf.add_face(TextFace(fourres), 2, position="float")

    # Add motifs/domains
    motifs = result_to_face(new_dom_dict_extra[str(leaf)])
    if motifs is not None:
        leaf.add_face(motifs, 0, "aligned")

tree.render(f"{stem}_domains.pdf", tree_style=everything, w=24, h=24, units="in");

0it [00:00, ?it/s]

In [ ]:
IDZ = ['A0A066ZRQ5', 'A0A173R3X4', 'A0A1Y4L5P0', 'A0A1Y4LM49']

In [ ]:
domain_dict = {}

# Iterate over leaves and query UniProt
# Assume that the last part of the string is the accession ID at UniProt
# (we also need to strip the final quote)
for leaf in tqdm(IDZ):
    accession = str(leaf).split()[-1][:-1]
    result = u.search(accession, frmt = 'tsv', columns = "ft_domain")
    reslen = u.search(accession, frmt = 'tsv', columns = "length")
    if (reslen is None) or (reslen.strip() == "Length"):
        length = None
    else:
        length = int(reslen.split()[-1].strip())
    domain_dict[str(leaf)] = (accession, result, length)